In [33]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [34]:
matches = pd.read_csv('nba_games_full.csv', index_col=0)

In [35]:
# cleaning up the data for use in the model
matches["date"] = pd.to_datetime(matches["Date"])
matches["venue_code"] = matches["Venue"].astype("category").cat.codes
matches["opp_code"] = matches["Opponent"].astype("category").cat.codes
matches["hour"] = matches["StartET"].str.replace(":.+", "", regex=True).astype("int")
matches["day_code"] = matches["date"].dt.dayofweek
matches["target"] = (matches["Result"] == "W").astype("int")

In [36]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [37]:
training_set = matches[matches["date"] < '2024-09-30']
test_set = matches[matches["date"] >= '2024-09-30']

In [38]:
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [39]:
# Predict && print metrics of initial model
rf.fit(training_set[predictors], training_set["target"])
predictions = rf.predict(test_set[predictors])
print("Accuracy:", accuracy_score(test_set["target"], predictions)) # whether it was right or wrong
print("Precision:", precision_score(test_set["target"], predictions)) # when it predicted win, how often was it correct

combined = pd.DataFrame(dict(actual=test_set["target"], prediction=predictions))
pd.crosstab(index=combined["actual"], columns=combined["prediction"])


Accuracy: 0.5606796116504854
Precision: 0.5632377740303541


prediction,0,1
actual,,
0,718,518
1,568,668


In [40]:
# showing how to group by team (prints all LAL games)
grouped_matches = matches.groupby("Team")
group = grouped_matches.get_group('LAL')
group

,Season,Date,StartET,Team,Opponent,Venue,Result,TeamPoints,OpponentPoints,Attendance,...,3P%,FTA,FTM,FT%,date,venue_code,opp_code,hour,day_code,target
Index,,,,,,,,,,,,,,,,,,,,,
1071,2023-24,2023-10-24,7:30p,LAL,Denver Nuggets,away,L,107,119,19842,...,0.345,20.0,15.0,0.750,2023-10-24,0,7,7,1,0
1072,2023-24,2023-10-26,10:00p,LAL,Phoenix Suns,home,W,100,95,18997,...,0.172,29.0,23.0,0.793,2023-10-26,1,23,10,3,1
1073,2023-24,2023-10-29,9:00p,LAL,Sacramento Kings,away,L,127,132,18198,...,0.333,34.0,26.0,0.765,2023-10-29,0,25,9,6,0
1074,2023-24,2023-10-30,10:30p,LAL,Orlando Magic,home,W,106,103,18997,...,0.296,18.0,14.0,0.778,2023-10-30,1,21,10,0,1
1075,2023-24,2023-11-01,10:00p,LAL,Los Angeles Clippers,home,W,130,125,18997,...,0.344,36.0,27.0,0.750,2023-11-01,1,12,10,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3622,2024-25,2025-04-06,3:30p,LAL,Oklahoma City Thunder,away,W,126,99,18203,...,0.550,17.0,12.0,0.706,2025-04-06,0,20,3,6,1
3623,2024-25,2025-04-08,8:00p,LAL,Oklahoma City Thunder,away,L,120,136,18203,...,0.450,37.0,28.0,0.757,2025-04-08,0,20,8,1,0
3624,2024-25,2025-04-09,7:30p,LAL,Dallas Mavericks,away,W,112,97,20841,...,0.333,21.0,16.0,0.762,2025-04-09,0,6,7,2,1


In [41]:
def rolling_averages(group, cols, new_cols):
  group = group.sort_values("date")
  rolling_stats = group[cols].rolling(3, closed='left').mean() # rolling average of last 3 games, excluding current game
  group[new_cols] = rolling_stats
  group = group.dropna(subset=new_cols) # drop rows where rolling averages are NaN (first 2 games of each team)
  return group

In [42]:
# create rolling average columns
cols = ["FGA", "FGM", "3PA", "3PM", "3P%", "FTA", "FTM", "FT%"]
new_cols = [f"rolling_{c}" for c in cols]

In [43]:
# test rolling_averages by showing rolling averages for LAL
rolling_averages(group, cols, new_cols)

,Season,Date,StartET,Team,Opponent,Venue,Result,TeamPoints,OpponentPoints,Attendance,...,day_code,target,rolling_FGA,rolling_FGM,rolling_3PA,rolling_3PM,rolling_3P%,rolling_FTA,rolling_FTM,rolling_FT%
Index,,,,,,,,,,,,,,,,,,,,,
1074,2023-24,2023-10-30,10:30p,LAL,Orlando Magic,home,W,106,103,18997,...,0,1,90.666667,40.000000,34.333333,10.000000,0.283333,27.666667,21.333333,0.769333
1075,2023-24,2023-11-01,10:00p,LAL,Los Angeles Clippers,home,W,130,125,18997,...,2,1,87.000000,40.333333,33.666667,9.333333,0.267000,27.000000,21.000000,0.778667
1076,2023-24,2023-11-04,7:00p,LAL,Orlando Magic,away,L,101,120,18846,...,5,0,89.000000,43.666667,34.666667,11.333333,0.324333,29.333333,22.333333,0.764333
1077,2023-24,2023-11-06,7:30p,LAL,Miami Heat,away,L,107,108,19725,...,0,0,84.333333,41.666667,29.666667,9.000000,0.302333,25.000000,20.000000,0.811000
1078,2023-24,2023-11-08,8:00p,LAL,Houston Rockets,away,L,94,128,18055,...,2,0,85.000000,42.333333,29.333333,9.000000,0.306333,23.666667,19.000000,0.813667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3622,2024-25,2025-04-06,3:30p,LAL,Oklahoma City Thunder,away,W,126,99,18203,...,6,1,84.666667,38.333333,44.666667,16.666667,0.373667,23.666667,21.333333,0.899667
3623,2024-25,2025-04-08,8:00p,LAL,Oklahoma City Thunder,away,L,120,136,18203,...,1,0,83.000000,42.000000,42.666667,18.666667,0.441000,22.666667,19.333333,0.835000
3624,2024-25,2025-04-09,7:30p,LAL,Dallas Mavericks,away,W,112,97,20841,...,2,1,81.000000,41.333333,40.333333,18.666667,0.463333,27.333333,22.000000,0.797333


In [44]:
matches_rolling = matches.groupby("Team").apply(lambda x: rolling_averages(x, cols, new_cols))
# matches_rolling = matches_rolling.droplevel("Team") # drop the extra index level added by groupby
matches_rolling.index = range(matches_rolling.shape[0]) # reset index
matches_rolling

C:\Users\Cr4zy\AppData\Local\Temp\ipykernel_43452\3264734097.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("Team").apply(lambda x: rolling_averages(x, cols, new_cols))


,Season,Date,StartET,Team,Opponent,Venue,Result,TeamPoints,OpponentPoints,Attendance,...,day_code,target,rolling_FGA,rolling_FGM,rolling_3PA,rolling_3PM,rolling_3P%,rolling_FTA,rolling_FTM,rolling_FT%
0,2023-24,2023-10-30,7:30p,ATL,Minnesota Timberwolves,home,W,127,113,15504,...,0,1,91.000000,42.666667,32.666667,10.666667,0.317333,28.333333,23.000000,0.812000
1,2023-24,2023-11-01,7:30p,ATL,Washington Wizards,home,W,130,121,15925,...,2,1,88.666667,45.666667,33.000000,13.666667,0.415667,23.333333,19.666667,0.854000
2,2023-24,2023-11-04,7:00p,ATL,New Orleans Pelicans,away,W,123,105,17237,...,5,1,90.333333,47.000000,33.000000,12.666667,0.384333,24.000000,21.333333,0.889333
3,2023-24,2023-11-06,8:00p,ATL,Oklahoma City Thunder,away,L,117,126,16486,...,0,0,90.333333,46.333333,34.333333,12.333333,0.363000,24.666667,21.666667,0.880667
4,2023-24,2023-11-09,9:30p,ATL,Orlando Magic,away,W,120,119,19986,...,3,1,95.666667,43.000000,38.333333,12.333333,0.318333,29.333333,25.000000,0.847333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4804,2024-25,2025-04-06,6:00p,WAS,Boston Celtics,away,L,90,124,19156,...,6,0,85.000000,38.333333,38.666667,14.000000,0.365667,15.666667,11.666667,0.754000
4805,2024-25,2025-04-08,7:00p,WAS,Indiana Pacers,away,L,98,104,16144,...,1,0,85.000000,38.666667,39.333333,13.666667,0.354000,13.000000,10.000000,0.780000
4806,2024-25,2025-04-09,7:00p,WAS,Philadelphia 76ers,home,L,103,122,17222,...,2,0,87.333333,37.000000,38.000000,11.666667,0.316333,12.000000,9.333333,0.783667
4807,2024-25,2025-04-11,8:00p,WAS,Chicago Bulls,away,L,89,119,21400,...,4,0,88.000000,35.666667,39.000000,11.000000,0.284000,18.333333,14.666667,0.788667


In [45]:
def make_predictions(data, predictors):
  training_set = data[data["date"] < '2024-09-30']
  test_set = data[data["date"] >= '2024-09-30']
  rf.fit(training_set[predictors], training_set["target"])
  predictions = rf.predict(test_set[predictors])
  combined = pd.DataFrame(dict(actual=test_set["target"], prediction=predictions), index=test_set.index)
  precision = precision_score(test_set["target"], predictions)
  return combined, precision

In [46]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [ ]:
combined

0.55668358714044